In [1]:
import pandas as pd
import os

# Load all CSV files from the "Preprocessed Data" folder
folder_path = r'Preprocessed_Data\node22c Oct23_Jan24'
dataframes = []

for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root, file)
            df = pd.read_csv(file_path)
            dataframes.append(df)

# Combine all dataframes into a single dataframe
combined_data = pd.concat(dataframes, ignore_index=True)


In [2]:
combined_data.tail()

,Date Time,Slave_Device1_CH1_FLOW m3/h,Slave_Device1_CH2_VELOCITY m/s,Slave_Device1_CH3_Pressure BAR
1130455,2023-10-30 22:04:06,74.0,0.50,1.25
1130456,2023-10-30 22:04:09,75.0,0.50,1.25
1130457,2023-10-30 22:04:13,74.5,0.50,1.25
1130458,2023-10-30 22:04:17,74.0,0.50,1.24
1130459,2023-10-30 22:45:41,68.5,0.46,1.26


In [3]:
combined_data.describe()

,Slave_Device1_CH1_FLOW m3/h,Slave_Device1_CH2_VELOCITY m/s,Slave_Device1_CH3_Pressure BAR
count,1.130460e+06,1.130460e+06,1.130460e+06
mean,5.966216e+01,4.002126e-01,1.146587e+00
std,2.946315e+01,1.978137e-01,3.545427e-01
min,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.700000e+01,3.200000e-01,1.180000e+00
50%,6.225000e+01,4.200000e-01,1.260000e+00
75%,7.450000e+01,5.000000e-01,1.340000e+00
max,2.490000e+02,2.510000e+00,2.510000e+00


In [4]:
# Convert 'Date Time' to datetime and extract time-based features
combined_data['Date Time'] = pd.to_datetime(combined_data['Date Time'])
combined_data['hour'] = combined_data['Date Time'].dt.hour
combined_data['day_of_week'] = combined_data['Date Time'].dt.dayofweek

# Rename columns for easier access
combined_data.rename(columns={
    'Slave_Device1_CH1_FLOW m3/h': 'flow',
    'Slave_Device1_CH3_Pressure BAR': 'pressure'
}, inplace=True)

# Create lag features for flow and pressure
combined_data['flow_lag1'] = combined_data['flow'].shift(1)
combined_data['pressure_lag1'] = combined_data['pressure'].shift(1)

# Calculate rolling mean for flow and pressure
combined_data['flow_roll_mean'] = combined_data['flow'].rolling(window=5).mean()
combined_data['pressure_roll_mean'] = combined_data['pressure'].rolling(window=5).mean()

# Drop rows with NaN values after creating lag and rolling features
combined_data.dropna(inplace=True)

# Define features and target variable
features = combined_data[['flow', 'pressure', 'hour', 'day_of_week', 
                          'flow_lag1', 'pressure_lag1', 
                          'flow_roll_mean', 'pressure_roll_mean']]

# labels = combined_data['burst_event']  # Assuming a 'burst_event' column for binary target


In [5]:
combined_data.columns

Index(['Date Time', 'flow', 'Slave_Device1_CH2_VELOCITY m/s', 'pressure',
       'hour', 'day_of_week', 'flow_lag1', 'pressure_lag1', 'flow_roll_mean',
       'pressure_roll_mean'],
      dtype='object')

In [6]:
import numpy as np

# Define thresholds
FLOW_THRESHOLD = 100  # Flow rate threshold in m³/s
PRESSURE_THRESHOLD = 3  # Pressure threshold in bar
FLOW_CHANGE_THRESHOLD = 10  # Threshold for a sudden increase in flow rate
PRESSURE_CHANGE_THRESHOLD = 0.5  # Threshold for a sudden increase in pressure

# Calculate change rates
combined_data['flow_change'] = combined_data['flow'].diff().abs()
combined_data['pressure_change'] = combined_data['pressure'].diff().abs()

# Define burst event based on threshold conditions
combined_data['burst_event'] = np.where(
    (combined_data['flow'] >= FLOW_THRESHOLD) |
    (combined_data['pressure'] >= PRESSURE_THRESHOLD) |
    (combined_data['flow_change'] >= FLOW_CHANGE_THRESHOLD) |
    (combined_data['pressure_change'] >= PRESSURE_CHANGE_THRESHOLD),
    1,  # Label as burst
    0   # No burst
)

# Drop rows with any NaN values in the dataset after feature engineering
combined_data.dropna(inplace=True)

# Now, you should have consistent row counts across all columns, allowing you to define features and labels
features = combined_data[['flow', 'pressure', 'hour', 'day_of_week', 
                          'flow_lag1', 'pressure_lag1', 
                          'flow_roll_mean', 'pressure_roll_mean']]
labels = combined_data['burst_event']


In [7]:
labels.describe()

count    1.130455e+06
mean     7.482032e-02
std      2.631013e-01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.000000e+00
Name: burst_event, dtype: float64

In [8]:
# labels = combined_data['burst_event']

In [8]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Define LightGBM dataset format
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Create a LightGBM dataset
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# Set parameters (tuned for faster training)
params = {
    'objective': 'binary',          # for binary classification
    'boosting_type': 'gbdt',        # traditional gradient boosting
    'n_estimators': 100,            # number of boosting rounds
    'max_depth': 10,                # limit depth to reduce complexity
    'num_leaves': 31,               # default for LightGBM, balance with max_depth
    'learning_rate': 0.1,           # faster training
    'n_jobs': -1                    # use all available cores
}

# Train the model
model = lgb.train(params, train_data, valid_sets=[test_data])

# Evaluate the model
y_pred = model.predict(X_test)
y_pred_binary = [1 if p > 0.5 else 0 for p in y_pred]  # convert probabilities to binary predictions
print(confusion_matrix(y_test, y_pred_binary))
print(classification_report(y_test, y_pred_binary))


c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Number of positive: 67698, number of negative: 836666
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.035606 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1365
[LightGBM] [Info] Number of data points in the train set: 904364, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.074857 -> initscore=-2.514368
[LightGBM] [Info] Start training from score -2.514368
[[209171     37]
 [   126  16757]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    209208
           1       1.00      0.99      1.00     16883

    accuracy                           1.00    226091
   macro avg       1.00      1.00      1.00    226091
weighted avg       1.00      1.00      1.00    226091



In [10]:
# from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import classification_report, confusion_matrix
# from sklearn.preprocessing import StandardScaler

# # Standardize features
# scaler = StandardScaler()
# features_scaled = scaler.fit_transform(features)

# # Split the dataset
# X_train, X_test, y_train, y_test = train_test_split(features_scaled, labels, test_size=0.2, random_state=42, shuffle=False)

# # Initialize and train the model with hyperparameter tuning
# model = RandomForestClassifier()
# param_grid = {
#     'n_estimators': [100, 200],
#     'max_depth': [None, 10, 20],
# }
# grid_search = GridSearchCV(model, param_grid, cv=5)
# grid_search.fit(X_train, y_train)

# # Model evaluation
# y_pred = grid_search.predict(X_test)
# print(confusion_matrix(y_test, y_pred))
# print(classification_report(y_test, y_pred))


In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam

# Example parameters
# TIME_STEPS = 30  # Number of time steps in each sequence
TIME_STEPS = 20
FEATURES = 3  # Number of features: FLOW, VELOCITY, PRESSURE

# Assuming `data` is your concatenated DataFrame with normalized values

# Prepare sequences
def create_sequences(data, time_steps=TIME_STEPS):
    sequences = []
    labels = []
    for i in range(len(data) - time_steps):
        seq = data.iloc[i:i + time_steps].values
        label = data.iloc[i + time_steps]['burst_event']  # Assuming 'Burst' is the target column
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

# Scale the data
scaler = MinMaxScaler()
combined_data[['flow', 'Slave_Device1_CH2_VELOCITY m/s', 'pressure']] = scaler.fit_transform(combined_data[['flow', 'Slave_Device1_CH2_VELOCITY m/s', 'pressure']])

# Split data into sequences
X, y = create_sequences(combined_data)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# LSTM Model
# Reduced time steps and batch size
BATCH_SIZE = 64  # Increase batch size

# Simplified LSTM Model
model = Sequential([
    LSTM(25, input_shape=(TIME_STEPS, FEATURES)),  # Reduce units and remove return_sequences
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train with reduced epochs for testing
history = model.fit(X_train, y_train, epochs=5, batch_size=BATCH_SIZE, validation_split=0.2)



In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')


In [ ]:
# Predict on new data
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)  # Convert probabilities to binary predictions
